# Working with AML Datasets & Data in general

In [ ]:
## Check core SDK version number
import azureml.core
import mlflow
import os

from azureml.core import (Datastore, Dataset, Environment, Experiment, ScriptRunConfig,
                          Workspace)
from azureml.core.authentication import InteractiveLoginAuthentication
from IPython.display import display



print("[INFO] SDK version:", azureml.core.VERSION)

## due to diferent tenant -> typically customer tenant
# interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")

ws = Workspace.from_config()
print("[SUCCESS] LOGGED IN: ",ws.name, ws.resource_group, ws.location, ws.subscription_id, sep=' @ ')

## set mlflow backend to AML
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

print("[INFO] MLFlow wired to AML:", "experiments.azureml.net" in mlflow.get_tracking_uri())

## Config

In [ ]:
aml_compute = "aml-cluster"
aml_ds = "aml_data"# "mmaadlsgen2_test"
# aml_dset = 'noa_weather'
# aml_dset = "oj_sample_data"
aml_dset = "diabetes_multiple"
aml_experiment = "mlflow-azureml"
loc_data = "data/demo_data"

In [ ]:
!rm -rf $loc_data

In [ ]:
## List all datastores registered in the current workspace
datastores = ws.datastores
for name, datastore in datastores.items():
    print(f"{name} ({datastore.datastore_type})")



In [ ]:
## set the datastore
ds = ws.datastores[aml_ds]
print(f"[INFO] Datastore: {ds.name}, type: {ds.datastore_type}")

## Data

### File Dataset

```python
from_files(path, validate=True)
```

In [ ]:
## create a FileDataset pointing to files in 'diabetes' folder and its subfolders recursively

# datastore_paths = [(ds, 'diabetes')]
# datastore_paths = [(ds, 'diabetes/diabetes0.csv'),(ds, 'diabetes/diabetes1.csv'),(ds, 'diabetes/diabetes2.csv')]
datastore_paths = [(ds, 'diabetes/diabetes*.csv')]
# datastore_paths = [(ds, 'ojs/Store140*.csv')]

fds = Dataset.File.from_files(path=datastore_paths)

In [ ]:
## show files matching criteria into paths
fds.to_path()

In [ ]:
## download just a sample of the data
fds.take(3).download(target_path=os.path.join("./",loc_data), overwrite=True)

In [ ]:
## check donwloaded files
tmppath = os.path.join("./",loc_data)
onlyfiles = [f for f in os.listdir(tmppath) if os.path.isfile(os.path.join(tmppath, f))]
print(onlyfiles)

### Tabular Dataset

```python
from_delimited_files(path, separator=',', header=<PromoteHeadersBehavior.ALL_FILES_HAVE_SAME_HEADERS: 3>, encoding=<FileEncoding.UTF8: 0>, quoting=False, infer_column_types=True, skip_rows=0, skip_mode=<SkipLinesBehavior.NO_ROWS: 0>, comment=None, include_path=False, archive_options=None, partition_format=None)

```

In [ ]:
## create a FileDataset pointing to files in 'weather' folder and its subfolders recursively

# datastore_paths = [(ds, 'diabetes')]
# datastore_paths = [(ds, 'diabetes/diabetes0.csv'),(ds, 'diabetes/diabetes1.csv'),(ds, 'diabetes/diabetes2.csv')]
datastore_paths = [(ds, 'diabetes/diabetes*.csv')]
# datastore_paths = [(ds, 'ojs/Store140*.csv')]

tds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

In [ ]:
pdf = tds.to_pandas_dataframe()

In [ ]:
pdf.info()

In [ ]:
## register Dataset into Workspace for reusability

wtds = tds.register(workspace=ws, name=aml_dset, description='Sample: Diabetes data from Azure Open Datasets',create_new_version=True)

In [ ]:
# wtds_profile = wtds.get_profile(ws)

### get data from existing dataset

In [ ]:
wtds = Dataset.get_by_name(ws, name=aml_dset)
wtds.to_pandas_dataframe().head(10)

## Training

In [ ]:
mlflow.set_experiment(aml_experiment)
